<a href="https://colab.research.google.com/github/letianzj/QuantResearch/blob/master/notebooks/python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Some advanced topics of Python

- [Numpy](#numpy)
- [Pandas](#pandas)
- [Other](#other)
- [Reference](#reference)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

## Numpy <a name="numpy"></a>


Python integer is not a pure integer. The PyOject_HEAD contains a reference count, the type and size information.

![image](./img/cint_vs_pyint.png)


Similarly, a Python list is more than just a list. The items is a pointer to a block of pointers, each of which in turn points to a full Python object like the Python integer. It is called dynamic-type. In contrast, numpy array is of fixed type where a gorup of homogeneous data are saved in a contiguous segment. 


![image](./img/array_vs_list.png)


- [Basic Attributes](#numpy_attributes)
- [Indexing and Slicing](#numpy_indexing)
- [ufunc](#numpy_ufunc)

### Basic Attributes <a name="numpy_attributes"></a>

In [2]:
x = np.full((3, 2), [1, 2], dtype=np.int32)
x

array([[1, 2],
       [1, 2],
       [1, 2]])

In [3]:
# 3 rows and 2 columns; size = 2x3 = 6; each int32 item has 4 bytes or 4x8=32bits
# Jump 2 items or 2x4=8 bytes to the next row; Jump 1 item or 4 bytes to the next column
print(x.shape, x.size, x.itemsize, x.strides)
print(x.__array_interface__['data'])         # memory address

(3, 2) 6 4 (8, 4)
(2192347358208, False)


In [4]:
# A reshape does not make a copy; it is a view by altering shape and strides
y = x.reshape(2,3)
print(y.shape, y.size, y.itemsize, y.strides)
print(y.__array_interface__['data'])
# So is a transpose
print((x.T).__array_interface__['data'])

(2, 3) 6 4 (12, 4)
(2192347358208, False)
(2192347358208, False)


In [5]:
# Numpy will force a copy if it becomes non-contiguous.
xt_flat = (x.T).reshape(6,)
print(xt_flat.shape, xt_flat.size, xt_flat.itemsize, xt_flat.strides)
print(xt_flat.__array_interface__['data'])

(6,) 6 4 (4,)
(2192347358240, False)


### Indexing and Slicing<a name="numpy_indexing"></a>

In [6]:
# indexing
print(x[0,0])

1


In [7]:
# slicing x[start:stop:step] is a non-copy view; by modifying the shape and strides
y = x[0:2, 0]
print(y.__array_interface__['data'][0], x.__array_interface__['data'][0])   # same memory address
y[0] = 10
print(x)    # x changes
y = np.array([5, 5])  # y points to another array, so x does not change
print(x)

2192347358208 2192347358208
[[10  2]
 [ 1  2]
 [ 1  2]]
[[10  2]
 [ 1  2]
 [ 1  2]]


In [8]:
# Boolean mask
np.sum(x < 2, axis=0)

array([2, 0])

In [9]:
# fancy indexing
row = np.array([0, 1, 2])
col = np.array([1, 0, 0])
y = x[row, col]
print(y)     # three elements: (0,1), (1,0), (2,0)
# fancy indexing returns a copy
y[2] = 5
print(x)       # x does not change

[2 1 1]
[[10  2]
 [ 1  2]
 [ 1  2]]


### ufunc<a name="numpy_ufunc"></a>

Python loop is interpretive therefore slow. By using NumPy's universal functions (ufuncs), you claim that the loop can be vectorized and pushed into the compiled layer.

In [10]:
big_array = np.random.rand(1_000_000)
%timeit sum(big_array)
%timeit np.sum(big_array)

96.1 ms ± 3.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.02 ms ± 34.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
# broadcasting
y = np.ones((3,1))
x+y     # y is broadcasted

array([[11.,  3.],
       [ 2.,  3.],
       [ 2.,  3.]])

ufunc advanced: aggregation

![image](./img/map_filter_reduce.jpg)

In [12]:
# map
def square(x):
    return x*x

x = [1, 2, 3, 4, 5]
print(list(map(square, x)))

vfunc = np.vectorize(square)
print(vfunc(x))

[1, 4, 9, 16, 25]
[ 1  4  9 16 25]


In [13]:
# filter
def check_even(number):
    if number % 2 == 0:
          return True  

    return False

even_numbers_iterator = filter(check_even, x)
print(list(even_numbers_iterator))

x = np.array(x)
mask = x%2 ==0
print(x[mask])

[2, 4]
[2 4]


In [14]:
# reduce
np.add.reduce(x)

15

## Pandas <a name="pandas"></a>

Pandas Series is an indexed numpy array. Index is immutable and ordered (but not necessarily sorted).

DataFrame consists of multiple column-oriented series that share an index. Each column has a dtype.

- [Grouping](#pandas_grouping)
- [Apply](#pandas_apply)
- [Merge](#pandas_merge)

In [52]:
np.random.seed(0)
df_orders = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=['BS', 'B', 'O', 'OS'])
df_orders['Sym'] = np.random.choice(['AAPL', 'AMZN'], 100, replace=True)
df_orders.index = pd.date_range("2020-01-01 09:30:00", periods=100, freq='ns', tz='US/Eastern')

df_trades = pd.DataFrame(np.random.randint(0,100,size=(100, 2)), columns=['P', 'S'])
df_trades['Sym'] = np.random.choice(['AAPL', 'AMZN'], 100, replace=True)
df_trades.index = pd.date_range("2020-01-01 09:30:00", periods=100, freq='ns', tz='US/Eastern') + pd.Timedelta(2, 'N')

df_trades.head(5)

,P,S,Sym
2020-01-01 09:30:00.000000002-05:00,10,94,AAPL
2020-01-01 09:30:00.000000003-05:00,91,43,AAPL
2020-01-01 09:30:00.000000004-05:00,63,31,AAPL
2020-01-01 09:30:00.000000005-05:00,20,70,AMZN
2020-01-01 09:30:00.000000006-05:00,9,60,AAPL


In [85]:
t1 = df_trades.index[0] + pd.Timedelta(1, 'N')
t2 = t1 + pd.Timedelta(3, 'N')
df_trades[t1:t2]       # slicing by index

,P,S,Sym
2020-01-01 09:30:00.000000003-05:00,10,43,AAPL
2020-01-01 09:30:00.000000004-05:00,63,31,AAPL
2020-01-01 09:30:00.000000005-05:00,20,70,AMZN
2020-01-01 09:30:00.000000006-05:00,9,60,AAPL


In [86]:
df_trades[1:3]         # slicing by row number

,P,S,Sym
2020-01-01 09:30:00.000000003-05:00,10,43,AAPL
2020-01-01 09:30:00.000000004-05:00,63,31,AAPL


In [88]:
df_trades[(df_trades.S > 20) & (df_trades.S < 25)]     # masking

,P,S,Sym
2020-01-01 09:30:00.000000014-05:00,39,24,AMZN
2020-01-01 09:30:00.000000034-05:00,7,21,AMZN
2020-01-01 09:30:00.000000094-05:00,77,21,AAPL
2020-01-01 09:30:00.000000097-05:00,25,21,AAPL


### Grouping

groupby: split/apply/combine

## Reference <a name="reference"></a>

* [Numpy QuickStart](https://numpy.org/doc/stable/user/quickstart.html)

* [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html)

* [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/)